
# Set up the notebook

In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

from choropleth_utils import ChoroplethUtilities
from stats_scraping_utils import StatsScrapingUtilities
from storage import Storage
import pandas as pd
import re
import os
import numpy as np

s = Storage()
ssu = StatsScrapingUtilities(s=s)

In [3]:

column_description_dict = s.load_object('column_description_dict')
us_stats_df = s.load_object('us_stats_df')

all_countries_df = s.load_object('all_countries_df').set_index('country_code', drop=True)
all_countries_df.country_name = all_countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
s.store_objects(all_countries_df=all_countries_df.reset_index(drop=False))

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\all_countries_df.pkl



----
# Get State/Country Cost of Living Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://meric.mo.gov/data/cost-living-data-series'
tables_list = ssu.get_page_tables(url)

[(1, (6, 8)), (0, (0, 9))]


In [5]:

body_opener_str = '</thead>\n  <tbody>\n  '
head_closer_str = '<th>Misc.</th>\n    </tr>\n    '
html_str = tables_list[0].to_html().replace(body_opener_str, '').replace(head_closer_str,
                                                                         head_closer_str + body_opener_str).replace('th>', 'td>')
tables_list = ssu.get_page_tables(html_str)

[(0, (52, 10))]


In [6]:

us_states_df = tables_list[0].copy()
# print(us_states_df.columns.tolist())
floats_list = ['cost_of_living_index', 'grocery_index', 'housing_index', 'utilities_index', 'transportation_index',
                'health_index', 'misc_index']
columns_list = ['cost_of_living_rank', 'state_name'] + floats_list
us_states_df.columns = ['deleteme'] + columns_list
us_states_df = us_states_df[columns_list]
for cn in floats_list:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
mask_series = us_states_df.state_name.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
cost_of_living_rank,1,2,3,4,5,6,7,8,9,10
state_name,Mississippi,Oklahoma,Kansas,Alabama,Iowa,Georgia,Indiana,Tennessee,Arkansas,Michigan
cost_of_living_index,83.099998,84.800003,85.400002,87.5,88.099998,88.199997,89.199997,89.400002,89.900002,89.900002
grocery_index,92.300003,94.0,91.699997,97.0,99.400002,94.099998,93.400002,94.5,92.900002,90.400002
housing_index,63.799999,68.800003,68.199997,68.900002,69.5,74.199997,75.900002,80.699997,76.699997,77.800003
utilities_index,90.0,95.199997,98.699997,101.099998,94.800003,91.199997,100.800003,94.099998,96.5,98.199997
transportation_index,89.099998,90.300003,94.199997,90.0,93.400002,91.400002,97.099998,87.900002,94.5,95.699997
health_index,97.400002,92.900002,102.800003,90.5,99.099998,94.300003,95.400002,91.699997,80.099998,97.0
misc_index,92.0,89.699997,90.400002,95.5,95.599998,96.699997,93.800003,93.800003,99.699997,96.599998


In [7]:

# Remove DC from the comparison so it doesn't skew the results
mask_series = (us_states_df.state_name.isin(['District of Columbia', '***']))
us_states_df = us_states_df[~mask_series]

In [8]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
print(states_list)
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])

['District of Columbia']



## Clean and prepare Countries dataset

In [9]:

url = 'https://www.numbeo.com/cost-of-living/rankings_by_country.jsp'
tables_list = ssu.get_page_tables(url)

[(1, (137, 8)), (0, (1, 2))]


In [10]:

countries_df = tables_list[1].copy()
# print(countries_df.columns.tolist())
floats_list = ['cost_of_living_index', 'rent_index', 'cost_of_living_plus_rent_index', 'groceries_index',
               'restaurant_price_index', 'local_purchasing_power_index']
columns_list = ['country_name'] + floats_list
countries_df.columns = ['cost_of_living_rank'] + columns_list
for cn in floats_list:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='float')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series][columns_list]
# countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

,0,1,2,3,4,5,6,7,8
country_name,Bermuda,Switzerland,Bahamas,Barbados,Iceland,Norway,Jersey,Singapore,Israel
cost_of_living_index,141.740005,110.339996,88.269997,87.07,86.589996,85.93,80.379997,79.089996,77.279999
rent_index,98.959999,50.209999,38.41,21.16,35.810001,30.549999,55.369999,65.089996,32.950001
cost_of_living_plus_rent_index,121.389999,81.730003,64.550003,55.709999,62.43,59.59,68.480003,72.43,56.189999
groceries_index,142.619995,113.349998,71.919998,81.730003,82.440002,81.309998,64.519997,71.360001,66.449997
restaurant_price_index,144.740005,104.300003,94.290001,74.5,88.639999,87.589996,89.230003,52.200001,83.919998
local_purchasing_power_index,82.339996,116.190002,44.220001,32.619999,82.620003,86.269997,68.720001,94.650002,73.540001


In [11]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
if countries_list:
    print(countries_list)
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

['Afghanistan', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua & Barbuda', 'Aruba', 'Benin', 'Bhutan', 'Bonaire, Sint Eustatius & Saba', 'Bouvet Island', 'British Indian Ocean Territory', 'British Virgin Islands', 'Burkina Faso', 'Burundi', 'Cayman Islands', 'Central African Republic', 'Chad', 'Christmas Island', 'Cocos (Keeling) Islands', 'Comoros', 'Cook Islands', 'Curaçao', "Côte d'Ivoire", 'DRC', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Federated States of Micronesia', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Gibraltar', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See', 'Isle of Man', 'Ivory Coast', 'Kiribati', 'Kosovo (Disputed Territory)', 'Laos', 'Lesotho', 'Liberia', 'Liechtenstein', 'Madagascar', 'Malawi', 'Mali', 'Marshall Islands', 'Martinique', 'M

,first_item,second_item,max_similarity
38,Gambia,Zambia,0.833333
58,Malawi,Mali,0.800000
13,British Virgin Islands,US Virgin Islands,0.769231
22,Cook Islands,Norfolk Island,0.769231
40,Greenland,Grenada,0.750000
16,Cayman Islands,Åland Islands,0.740741
84,Sint Maarten,St. Martin,0.727273
85,Solomon Islands,Åland Islands,0.714286
3,Angola,Anguilla,0.714286
80,San Marino,St. Martin,0.700000


In [12]:

# These countries cause redditors to make hurtful comments *sniff*
country_set = set(all_countries_df.country_name) - set(ssu.derisable_countries_list)
mask_series = countries_df.country_name.isin(country_set)
countries_df = countries_df[mask_series]

In [13]:

# Normalize the index so that the US is 100
mask_series = (countries_df.country_name == 'USA')
groceries_index = countries_df[mask_series].groceries_index.squeeze()
countries_df['normalized_groceries_index'] = countries_df.groceries_index.map(lambda x: 100*x/groceries_index)
mask_series = (countries_df.country_name == 'USA')
countries_df[mask_series].T

,14
country_name,USA
cost_of_living_index,69.919998
rent_index,46.860001
cost_of_living_plus_rent_index,58.950001
groceries_index,70.059998
restaurant_price_index,65.75
local_purchasing_power_index,99.879997
normalized_groceries_index,100.0



## Prepare for and Create Choropleth

In [14]:

equivalence_column_name = 'Country_Equivalent_Groceries_Index'
states_target_column_name = 'grocery_index'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'normalized_groceries_index', us_states_df,
                              st_col_name=states_target_column_name,
                              st_col_explanation='Groceries Index (2022)',
                              equivalence_column_name=equivalence_column_name, verbose=True)


Albania (39.47) is close to the normalized groceries index of Texas (89.40)
Algeria (39.05) is close to the normalized groceries index of Texas (89.40)
Argentina (38.30) is close to the normalized groceries index of Texas (89.40)
Armenia (48.44) is close to the normalized groceries index of Texas (89.40)
Australia (103.84) is close to the normalized groceries index of Minnesota (103.90)
Austria (82.84) is close to the normalized groceries index of Texas (89.40)
Azerbaijan (36.63) is close to the normalized groceries index of Texas (89.40)
Bahamas (102.65) is close to the normalized groceries index of Arizona (102.60)
Bahrain (62.03) is close to the normalized groceries index of Texas (89.40)
Bangladesh (39.89) is close to the normalized groceries index of Texas (89.40)
Barbados (116.66) is close to the normalized groceries index of California (117)
Belarus (34.50) is close to the normalized groceries index of Texas (89.40)
Belgium (75.89) is close to the normalized groceries index of 

In [15]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(
    numeric_column_name=states_target_column_name, string_column_name=equivalence_column_name,
    one_country_df=ssu.us_stats_df, cmap='summer')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_grocery_index_Country_Equivalent_Groceries_Index.svg
